<a href="https://colab.research.google.com/github/eddes/buildingphysics/blob/feature%2Fpipenv_and_notebooks/notebooks/chapter_2/PID_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PID controller

An illustration of Proportional-Integral-Derivative controler. In this example, a water draining tank has its water volume controlled by a PID. The associated equations are detailed in the book in 3.1.

The water height in the tank follows the equation 2.24 :
$ h^+ = h + \frac{\Delta t}{S_r} (Q - S_s \sqrt{2gh})$  with $Q$ the water inflow controled by the PID, $S_r$ the tank surface, $S_s$ the outlet surface.

Some dependencies import and plot color definition.



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
rouge_A='#C60C2E'
vert1_A='#005157'
vert2_A='#627D77'
vert3_A='#9EB28F'
vert4_A='#C5E5A4'
medblue='royalblue'
gris1_A='#595A5C'

colors=[rouge_A, vert1_A, vert2_A, vert3_A, vert4_A, medblue, gris1_A]

Helper function allowing to change the valve behaviour.


In [0]:
def fc_valve_curve(characteristic, valve_pos,Qmax):
    if characteristic == "linear":
        Qalim = Qmax * valve_pos
    elif characteristic == "equal_pct":
        Qalim = Qmax * np.exp(3.5 * (valve_pos - 1))
    elif characteristic == "quadratic":
        Qalim = Qmax * valve_pos**2
    return Qalim

Constant definitions and valve model selection.


In [0]:

#valve characteristic ="equal_pct" or "quadratic" or "linear"
characteristic="linear" 

# Geometrical data
Pi = math.pi # pi approximation
Dr = 1 #m the tank diameter
Ds = 0.04 #m the water outlet diameter
Sr = Pi * Dr**2 / 4 #m² tank surface 
Ss = Pi * Ds**2 / 4 #m² water outlet surface
H0 = 1 #m tank height

B = Ss / Sr * np.sqrt(2 * 9.81) # B, constant in the equation


The PID is defined by gains for proportionnal, derivative and integral corrections. Initial values are also setup for the simulation.

In [0]:
# PID parameters
prop_band = 0.5 # m
KP = 1 / prop_band # proportional gain 
Tn = 15 # Integration time
Td = 0 # Derivation time

Qmax = 10 # L/s supply flowrate
Qmax = Qmax / 1000 #conversion to m3/s

# initial values
h = H0 #m, reference water height value
Qalim = 0  # initial value for inflow rate
sim_time= 600 # simulation duration s

# time and timestep
t = 0
dt = 0.05 # [s] time stem /sampling rate

#Reset values
sum_error = 0
delta_error = 0
valve_position = 0
deltaT_previous = 0

pcent_P,pcent_I,pcent_D=[],[],[]
time,height=[],[]
v_pos=[]

For each timestep, the proportionnal, derivative and integral action are computed. The equation has been transformed as : 
$ h^+ = h + dt (A - B \sqrt{h})$ with $A =  \frac{Q}{S_r}$ and $B =  \frac{S_s  \sqrt{2g}}{Sr}$ .

Given the error $e$, difference between the measure output

- Proportionnal error is defined as $P_e(t+dt) = H_0 - h(t)$
- Integral error is defined as $I_e(t+dt) = \frac{dt}{Tn} (H_0 - h(t)) + I_e(t)$
- Derivative error is defined as $D_e(t+dt) = \frac{(H_0 - h(t)) - D_e(t)}{dt} $


In [0]:
#Euler explicit for water height
A = Qalim / Sr #compute A 
h = dt * (A - B * np.sqrt(h)) + h
#proportionnal error
prop_error = (H0 - h)
#integral error
sum_error = dt / Tn * prop_error + sum_error
#derivative error
delta_error = Td * (prop_error - deltaT_previous) / dt
#computation of the valve position with PID
valve_position = KP * (prop_error + sum_error +  delta_error)


The valve opening value is then updated.

In [0]:
if valve_position < 0:
    valve_position = 0
    Qalim = 0
elif valve_position > 1:
    valve_position = 1
    Qalim = Qmax
else:
    Qalim=fc_valve_curve(characteristic, valve_position,Qmax)

An interactive plot is available here with PID gain and $B_m$ ($=B x 1000$) value to be modified. As a reminder,
$B $ is proportional to $ S_s/S_r$.
Integral and derivative components of the PID can also be modified with the edition of Tn and Td values.

In [0]:
def plot_water_tank_evolution(KP, Qmax_user,Tn,Td, sim_time,):
    Qmax=Qmax_user/1000 # convert to m3/s
    H0 = 1
    h = H0
    Qalim = 0  # initial value for inflow rate
    # time and timestep
    t = 0
    dt = 0.05 # [s] time stem /sampling rate

    #Reset values
    sum_error = 0
    delta_error = 0
    valve_position = 0
    deltaT_previous = 0

    pcent_P,pcent_I,pcent_D=[],[],[]
    time,height=[],[]
    v_pos=[]
    while t <= sim_time:
        #Euler explicit for water height
        A = Qalim / Sr #compute A 
        h = dt * (A - B * np.sqrt(h)) + h
        #proportionnal error
        prop_error = (H0 - h)
        #integral error
        sum_error = dt / Tn * prop_error + sum_error
        #derivative error
        delta_error = Td * (prop_error - deltaT_previous) / dt
        #computation of the valve position with PID
        valve_position = KP * (prop_error + sum_error +  delta_error)

        #Control for valve opening
        if valve_position < 0:
            valve_position = 0
            Qalim = 0
        elif valve_position > 1:
            valve_position = 1
            Qalim = Qmax
        else:
            Qalim=fc_valve_curve(characteristic, valve_position,Qmax)
        v_pos.append(valve_position)
        # let's see and plot who does what at each time step
        if valve_position > 0:
            pcent_P.append(abs (prop_error) * KP / valve_position)
            pcent_I.append( sum_error + KP * prop_error * (dt / Tn) / valve_position)
            pcent_D.append(KP * ( delta_error ) / valve_position)
        else:
            pcent_P.append(0)
            pcent_I.append(0)
            pcent_D.append(0)
        deltaT_previous = prop_error
        t+=dt
        time.append(t)
        height.append(h)

    plt.subplot(121)
    plt.xlabel("Time [s]")
    plt.ylabel("Water height [m]")
    plt.plot(time, height,color=colors[-2],linestyle="-",alpha=1,marker='')
    plt.plot(time, np.ones(len(time))*H0,color=colors[-1],linestyle="--",alpha=0.85,marker='')

    plt.subplot(122)
    plt.xlabel("Time [s]")
    plt.ylabel("Share of P, I and D in the valve opening [-]")
    plt.plot(time, pcent_P,color=colors[0],linestyle="-",alpha=1,marker='',label="P")
    plt.plot(time, pcent_I,color=colors[1],linestyle="-",alpha=1,marker='',label="I")
    if Td != 0:
        plt.plot(time, pcent_D,color=colors[2],linestyle="-",alpha=1,marker='',label="D")
    plt.plot(time, v_pos,color=colors[-1],linestyle="--",alpha=0.95,marker='',label="valve position")

    plt.legend()
    plt.tight_layout()

In [25]:
plt.rcParams["figure.figsize"] = (12,5)

interact_manual(plot_water_tank_evolution, KP = widgets.FloatSlider(value=2,    min=0.01,
                                               max=10,
                                               step=0.1),
                
                Qmax_user = widgets.FloatSlider(value=9,    min=1,
                                               max=12,
                                               step=0.5),
                Td = widgets.IntSlider(value=0,    min=0,
                                               max=100,
                                               step=1),
                Tn = widgets.IntSlider(value=15,    min=0,
                                               max=100,
                                               step=1),
                sim_time = widgets.IntSlider(value=600,    min=50,
                                               max=5000,
                                               step=50),
                 )

interactive(children=(FloatSlider(value=2.0, description='KP', max=10.0, min=0.01), FloatSlider(value=9.0, des…

<function __main__.plot_water_tank_evolution>

What happens when the derivative component is increased ? When the proportionnal component is increased ? 